In [2]:
import torch

print(torch.__version__)

2.4.0


In [3]:
torch.cuda.is_available() # True 이면 GPU가 잡혀있는 것

# Mac은 torch.backends.mps.is_available()
# tensorflow 와는 다르게 GPU를 우리가 지정할 수 있다.

True

In [4]:
import numpy as np

### 3. 파이토치 텐서 선언하기

In [9]:
t = torch.FloatTensor([0., 1., 2., 3., 4., 5., 6.]) # numpy array의 자식 class라서 메서드 사용 가능
print(t)

tensor([0., 1., 2., 3., 4., 5., 6.])


In [8]:
print(t.dim()) # rank (순위를 표현하기도 하지만 행렬의 복잡도를 표현하기도 한다. 높을 수록 복잡)
# dim : 차원, rank라고 하기도 함

1


In [10]:
t.shape

torch.Size([7])

In [11]:
t.size()

torch.Size([7])

In [12]:
print(t[0], t[1], t[-1]) # Element

tensor(0.) tensor(1.) tensor(6.)


In [13]:
print(t[2:5], t[4:-1]) # Slicing

tensor([2., 3., 4.]) tensor([4., 5.])


### 2차원 텐서

In [14]:
t = torch.FloatTensor([[1., 2., 3.],
                       [4., 5., 6.],
                       [7., 8., 9.],
                       [10., 11., 12.]
                      ])
print(t)

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])


In [15]:
print(t.dim()) # rank
print(t.size()) # shape

2
torch.Size([4, 3])


In [18]:
t[2, 1] # 행을 먼저 지정하고 열을 지정하면 특정 값을 가져올 수 있다.

tensor(8.)

In [19]:
t[2]

tensor([7., 8., 9.])

In [21]:
t[:, -1]

tensor([ 3.,  6.,  9., 12.])

In [22]:
# 브로드 캐스팅 pass

### 4) 최대(Max)와 아그맥스(ArgMax)

In [23]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [24]:
t.max()

tensor(4.)

In [27]:
t.max(dim=0) # 차원 하나 제거 후 나머지 애들을 표현, 덩어리 채로 , 행의 인덱스가 반환 되는 것 ?

torch.return_types.max(
values=tensor([3., 4.]),
indices=tensor([1, 1]))

In [34]:
# 10개의 사물 분류 모델
result = torch.FloatTensor([0.1, 0.04, 0.5, 0.1, 0.1, 0.01, 0.01, 0.01, 0.87, 0.1])
names = ['cat', 'dog', 'lion', 'tiger', 'worf', 'fox', 'snake', 'rabbit', 'horse', 'zebra']
result.argmax()
names[result.argmax()]

'horse'

### 5) 뷰(View) - 원소의 수를 유지하면서 텐서의 크기 변경. 매우 중요!

In [36]:
t = np.array([[[0, 1, 2],
               [3, 4, 5]],
              [[6, 7, 8],
               [9, 10, 11]]])
ft = torch.FloatTensor(t)
ft.shape

torch.Size([2, 2, 3])

In [37]:
# ft.view(list 형식의 차원 shape)
reshaped_ft = ft.view([3, 4])
reshaped_ft.shape

torch.Size([3, 4])

In [38]:
reshaped_ft = ft.view([4, 3])
reshaped_ft.shape

torch.Size([4, 3])

In [39]:
reshaped_ft = ft.view([3, -1]) # -1 : 나는 모르겠으니까 알아서 잡아달라는 뜻
reshaped_ft.shape

torch.Size([3, 4])

In [41]:
reshaped_ft = ft.view([5, -1])
reshaped_ft.shape

# 5로는 12를 깔끔하게 나누어 떨어지지 않기 때문에 안 됨

RuntimeError: shape '[5, -1]' is invalid for input of size 12

In [42]:
reshaped_ft = ft.view([4, 3, 1])
reshaped_ft.shape

torch.Size([4, 3, 1])

In [44]:
reshaped_ft = ft.view([2, 2, 1, 3]) # 숫자만 맞으면 4차원도 가능
reshaped_ft.shape

torch.Size([2, 2, 1, 3])

In [45]:
t = np.ones([128, 128])
tf = torch.FloatTensor(t)
tf.shape

torch.Size([128, 128])

In [47]:
tf = tf.view([128, 128, 1]) # 행렬 숫자의 차원을 3차원으로 맞춰준다.
tf.shape

torch.Size([128, 128, 1])

In [50]:
# 차원 하나를 제거해주는 것 : 스퀴즈

tf = tf.squeeze()
tf.shape

torch.Size([128, 128])

In [52]:
tf = tf.view([-1, 1]) # 앞은 얼마인지 모르지만 뒤는 1로 맞춰줘라
tf.shape

torch.Size([16384, 1])

In [53]:
tf = tf.squeeze()
tf.shape

torch.Size([16384])

In [55]:
# 0번 째 인덱스에 하나의 차원을 추가하고 싶다면?

tf = tf.unsqueeze(0) # 자유자재로 사용할 수 있어야 한다. 원하는 자리에 차원 1을 추가해줌ㄴ
tf.shape

torch.Size([1, 16384])

### 파이토치로 선형회귀 구현

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [79]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [80]:
x_train.shape, y_train.shape

(torch.Size([3, 1]), torch.Size([3, 1]))

In [81]:
# y = wx + b 에서 w, b를 알고 싶은 것, 그러기 위해서는 별도의 특수한 공간에 넣어주어야 한다. 
# 이런 파라미터의 특징은 값이 학습을 통해서 변화한다.
# 바뀌는 행렬이라는 점을 선언해줘야 한다.

In [82]:
W = torch.zeros(1, requires_grad=True)

# 그냥 torch.zeros는 그냥 일반적인 0으로 채워지는 행렬이 만들어지지만 
# requires_grad=True 를 하면 학습을 통해 역전파 이후 값이 바뀐다.
# 파라미터 값들은 requires_grad=True를 반드시 켜줘야 한다 !!! 중요
# 가중치가 학습 가능하다는 선언

In [83]:
b = torch.zeros(1, requires_grad=True)

In [84]:
'''
현재 식의 상태
y = 0 * x + 0
'''

y_pred = x_train * W + b

In [85]:
y_pred
# grad_fn=<AddBackward0>
# 이 값은 기울기 계산을 통해서 조정할 수 있다?

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)

In [86]:
loss = torch.mean((y_train - y_pred) ** 2) # MSE : 평균 제곱 손실
print(loss)

tensor(18.6667, grad_fn=<MeanBackward0>)


In [87]:
optimizer = optim.SGD([W, b], lr=0.001) # optim 을 불러오고 적용 대상과 학습률 설정
# optim.SGD(model.parameters(), lr=0.001) : 일종의 compile 단계

In [88]:
# 한 바퀴를 돌릴 때 마다 grad를 초기화 해야 한다.

optimizer.zero_grad()

# 우리가 손실값을 만들었을 때 requires_grad=True를 넣었으면 아래 태그를 넣어줘야 한다. 학습 반영하도록 !

loss.backward() 

optimizer.step() # : 파라미터 업데이트

# -> 손실 계산 후 역전파 하는 과정

In [92]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 20 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    # if epoch % 100 == 0:
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
    ))


Epoch    0/20 W: 0.187, b: 0.080 Cost: 18.666666
Epoch    1/20 W: 0.353, b: 0.151 Cost: 14.770963
Epoch    2/20 W: 0.500, b: 0.214 Cost: 11.691541
Epoch    3/20 W: 0.632, b: 0.270 Cost: 9.257346
Epoch    4/20 W: 0.749, b: 0.319 Cost: 7.333169
Epoch    5/20 W: 0.853, b: 0.363 Cost: 5.812135
Epoch    6/20 W: 0.945, b: 0.401 Cost: 4.609764
Epoch    7/20 W: 1.028, b: 0.435 Cost: 3.659278
Epoch    8/20 W: 1.101, b: 0.466 Cost: 2.907896
Epoch    9/20 W: 1.166, b: 0.492 Cost: 2.313895
Epoch   10/20 W: 1.224, b: 0.516 Cost: 1.844294
Epoch   11/20 W: 1.276, b: 0.536 Cost: 1.473027
Epoch   12/20 W: 1.322, b: 0.555 Cost: 1.179487
Epoch   13/20 W: 1.363, b: 0.571 Cost: 0.947386
Epoch   14/20 W: 1.400, b: 0.585 Cost: 0.763851
Epoch   15/20 W: 1.433, b: 0.597 Cost: 0.618704
Epoch   16/20 W: 1.462, b: 0.608 Cost: 0.503902
Epoch   17/20 W: 1.488, b: 0.617 Cost: 0.413086
Epoch   18/20 W: 1.511, b: 0.625 Cost: 0.341229
Epoch   19/20 W: 1.531, b: 0.632 Cost: 0.284360
Epoch   20/20 W: 1.550, b: 0.638 Cost

### 2. 다중 선형 회귀 구현하기

In [104]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 여기에 들어가는 가중치의 개수는 3개 : x1, x2, x3 로 구성되어 있기 때문


In [105]:
model = nn.Linear(3, 1) # parameter = input * output + 1 (+1은 bias)
# model.parameters() # 여기까지만 하면 <generator object Module.parameters at 0x7fb2a029af80> 이렇게 뜨는데 generator니까 list에 담아줘야 됨.
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3466,  0.0233, -0.4087]], requires_grad=True),
 Parameter containing:
 tensor([0.0657], requires_grad=True)]

In [106]:
optimizer = optim.SGD(model.parameters(), lr=1e-2)

epochs = 500

for epoch in range(epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train) # 순전파

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epochs % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch    0/500 Cost: 30929.994141
Epoch    1/500 Cost: 5964451840.000000
Epoch    2/500 Cost: 1150190899691520.000000
Epoch    3/500 Cost: 221804040866601369600.000000
Epoch    4/500 Cost: 42772926652312180070809600.000000
Epoch    5/500 Cost: 8248376952334104359529831464960.000000
Epoch    6/500 Cost: 1590625789280878258983536989799710720.000000
Epoch    7/500 Cost: inf
Epoch    8/500 Cost: inf
Epoch    9/500 Cost: inf
Epoch   10/500 Cost: inf
Epoch   11/500 Cost: inf
Epoch   12/500 Cost: inf
Epoch   13/500 Cost: inf
Epoch   14/500 Cost: inf
Epoch   15/500 Cost: nan
Epoch   16/500 Cost: nan
Epoch   17/500 Cost: nan
Epoch   18/500 Cost: nan
Epoch   19/500 Cost: nan
Epoch   20/500 Cost: nan
Epoch   21/500 Cost: nan
Epoch   22/500 Cost: nan
Epoch   23/500 Cost: nan
Epoch   24/500 Cost: nan
Epoch   25/500 Cost: nan
Epoch   26/500 Cost: nan
Epoch   27/500 Cost: nan
Epoch   28/500 Cost: nan
Epoch   29/500 Cost: nan
Epoch   30/500 Cost: nan
Epoch   31/500 Cost: nan
Epoch   32/500 Cost: nan
E

In [107]:
new_x = torch.FloatTensor([[73, 80, 75]])
new_y = model(new_x)

print(list(model.parameters()))
print(new_y)

[Parameter containing:
tensor([[nan, nan, nan]], requires_grad=True), Parameter containing:
tensor([nan], requires_grad=True)]
tensor([[nan]], grad_fn=<AddmmBackward0>)


In [108]:
# class로 모델 만드는 법
'''
1. 상속을 받야야 됨. nn.Module
2. def __init__(우리가 쓰고 싶은 Linear의 이름을 쭉 써주면 된다.), def forward 반드시 넣어줘야 한다.
'''

'\n1. 상속을 받야야 됨. nn.Module\n2. def __init__(우리가 쓰고 싶은 Linear의 이름을 쭉 써주면 된다.), def forward 반드시 넣어줘야 한다.\n'

In [149]:
# Subclass 모델

class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
    def __init__(self): #
        super().__init__()
        self.linear1 = nn.Linear(3, 3)
        self.linear2 = nn.Linear(3, 1)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [150]:
model = LinearRegressionModel()

In [151]:
list(model.parameters()) # bias까지 총 10개의 파라미터

[Parameter containing:
 tensor([[ 0.5548,  0.3410, -0.0780],
         [-0.2435,  0.2367, -0.0652],
         [ 0.1537,  0.4814,  0.1286]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4232,  0.1205, -0.1332], requires_grad=True),
 Parameter containing:
 tensor([[-0.4916, -0.2149,  0.4012]], requires_grad=True),
 Parameter containing:
 tensor([0.2917], requires_grad=True)]

In [152]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [153]:
optimizer  = optim.Adam(model.parameters(), lr = 4e-3) # 손실이 잘 줄어들지 않는다면 lr을 키워준다. 근데 또 너무 크면 막판에 잘 안 줄어들 수도 있음

In [154]:
epochs = 1000
for epoch in range(epochs + 1):
    # H(x) 계산
    prediction = model(x_train) # 순전파

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epochs % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch    0/1000 Cost: 32498.234375
Epoch    1/1000 Cost: 31971.839844
Epoch    2/1000 Cost: 31449.869141
Epoch    3/1000 Cost: 30932.394531
Epoch    4/1000 Cost: 30419.474609
Epoch    5/1000 Cost: 29911.175781
Epoch    6/1000 Cost: 29407.558594
Epoch    7/1000 Cost: 28908.671875
Epoch    8/1000 Cost: 28414.558594
Epoch    9/1000 Cost: 27925.265625
Epoch   10/1000 Cost: 27440.818359
Epoch   11/1000 Cost: 26961.250000
Epoch   12/1000 Cost: 26486.587891
Epoch   13/1000 Cost: 26016.839844
Epoch   14/1000 Cost: 25552.035156
Epoch   15/1000 Cost: 25092.167969
Epoch   16/1000 Cost: 24637.246094
Epoch   17/1000 Cost: 24187.271484
Epoch   18/1000 Cost: 23742.240234
Epoch   19/1000 Cost: 23302.140625
Epoch   20/1000 Cost: 22866.962891
Epoch   21/1000 Cost: 22436.691406
Epoch   22/1000 Cost: 22011.300781
Epoch   23/1000 Cost: 21590.779297
Epoch   24/1000 Cost: 21175.097656
Epoch   25/1000 Cost: 20764.226562
Epoch   26/1000 Cost: 20358.146484
Epoch   27/1000 Cost: 19956.822266
Epoch   28/1000 Cost

In [155]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import TensorDataset, Dataset, DataLoader

In [156]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [159]:
dataset = TensorDataset(x_train, y_train) # 1행의 답은 152, 2행의 답은 182, 3행의 답은 180 이라고 매칭시켜주는 부분

In [162]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True) # DataLoader는 반드시 dataset이 들어가야 한다.

In [164]:
for i, data in enumerate(dataloader):
    print(i, data)

# batch size가 2인데 데이터가 5개 이므로 마지막 한 개는 깍두기 취급해서 버리고 간다.

0 [tensor([[73., 80., 75.],
        [73., 66., 70.]]), tensor([[152.],
        [142.]])]
1 [tensor([[ 96.,  98., 100.],
        [ 93.,  88.,  93.]]), tensor([[196.],
        [185.]])]
2 [tensor([[89., 91., 90.]]), tensor([[180.]])]


In [166]:
# Subclass 모델

class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
    def __init__(self): #
        super().__init__()
        self.linear1 = nn.Linear(3, 3)
        self.linear2 = nn.Linear(3, 1)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x
    
model = LinearRegressionModel()
print(model) # model 구조를 볼 수도 있다.

LinearRegressionModel(
  (linear1): Linear(in_features=3, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
)


In [177]:
optimizer = optim.SGD(model.parameters(), lr = 5e-4)

epochs = 1000
for epoch in range(epochs + 1):
    for idx, batch in enumerate(dataloader):
        x, y_true = batch
        y_pred = model(x)

        loss = F.mse_loss(y_pred, y_true)

        # 역전파
        if idx == 0: # epoch 마다 작동
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 50 == 0:
            print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(epoch, epochs, idx, len(dataloader), loss.item()))

Epoch    0/1000 Batch 0/3 Cost: 29488.515625
Epoch    0/1000 Batch 1/3 Cost: 16014586.000000
Epoch    0/1000 Batch 2/3 Cost: 21764558.000000
Epoch   50/1000 Batch 0/3 Cost: 33742.683594
Epoch   50/1000 Batch 1/3 Cost: 25690.494141
Epoch   50/1000 Batch 2/3 Cost: 21707.312500
Epoch  100/1000 Batch 0/3 Cost: 30849.933594
Epoch  100/1000 Batch 1/3 Cost: 24540.828125
Epoch  100/1000 Batch 2/3 Cost: 16714.289062
Epoch  150/1000 Batch 0/3 Cost: 21475.070312
Epoch  150/1000 Batch 1/3 Cost: 22848.898438
Epoch  150/1000 Batch 2/3 Cost: 27135.181641
Epoch  200/1000 Batch 0/3 Cost: 24020.783203
Epoch  200/1000 Batch 1/3 Cost: 20697.873047
Epoch  200/1000 Batch 2/3 Cost: 15453.290039
Epoch  250/1000 Batch 0/3 Cost: 23620.595703
Epoch  250/1000 Batch 1/3 Cost: 18182.208984
Epoch  250/1000 Batch 2/3 Cost: 11519.321289
Epoch  300/1000 Batch 0/3 Cost: 21647.310547
Epoch  300/1000 Batch 1/3 Cost: 16468.804688
Epoch  300/1000 Batch 2/3 Cost: 10153.723633
Epoch  350/1000 Batch 0/3 Cost: 15483.580078
Epoc

### 커스텀 데이터셋

In [167]:
'''class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):
  데이터셋의 전처리를 해주는 부분 (데이터 경로를 지정하는 부분)

  def __len__(self):
  데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
  (데이터 전체 길이가 필요함 어디였지..)

  def __getitem__(self, idx): 
  데이터셋에서 특정 1개의 샘플을 가져오는 함수
  (인덱스 번호를 넣어주면 n 번째 데이터를 가져올 수 있음)'''

'class CustomDataset(torch.utils.data.Dataset): \n  def __init__(self):\n  데이터셋의 전처리를 해주는 부분 (데이터 경로를 지정하는 부분)\n\n  def __len__(self):\n  데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분\n\n  def __getitem__(self, idx): \n  데이터셋에서 특정 1개의 샘플을 가져오는 함수'

In [168]:
# Dataset 상속
class CustomDataset(Dataset): 
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴 (이중 리스트 처리 위해)
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y


In [171]:
dataset = CustomDataset()
dataset

In [173]:
dataset.__len__()

5

In [174]:
dataset.__getitem__(0)

(tensor([73., 80., 75.]), tensor([152.]))

In [175]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [176]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):

    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))


Epoch    0/20 Batch 1/3 Cost: 24211.177734
Epoch    0/20 Batch 2/3 Cost: 39606.726562
Epoch    0/20 Batch 3/3 Cost: 38482.238281
Epoch    1/20 Batch 1/3 Cost: 30612.113281
Epoch    1/20 Batch 2/3 Cost: 34329.390625
Epoch    1/20 Batch 3/3 Cost: 36235.042969
Epoch    2/20 Batch 1/3 Cost: 24211.177734
Epoch    2/20 Batch 2/3 Cost: 40730.328125
Epoch    2/20 Batch 3/3 Cost: 36235.042969
Epoch    3/20 Batch 1/3 Cost: 37358.640625
Epoch    3/20 Batch 2/3 Cost: 32860.203125
Epoch    3/20 Batch 3/3 Cost: 25680.365234
Epoch    4/20 Batch 1/3 Cost: 39606.726562
Epoch    4/20 Batch 2/3 Cost: 24211.177734
Epoch    4/20 Batch 3/3 Cost: 38482.238281
Epoch    5/20 Batch 1/3 Cost: 32860.203125
Epoch    5/20 Batch 2/3 Cost: 32081.300781
Epoch    5/20 Batch 3/3 Cost: 36235.042969
Epoch    6/20 Batch 1/3 Cost: 34329.390625
Epoch    6/20 Batch 2/3 Cost: 37358.640625
Epoch    6/20 Batch 3/3 Cost: 22741.990234
Epoch    7/20 Batch 1/3 Cost: 29488.515625
Epoch    7/20 Batch 2/3 Cost: 34329.390625
Epoch    7/